# 第13章 搜索

## 13.1 接口

根据上一章的问题：生成`[0, maxval]`范围内$m$个随机整数的有序序列，不允许重复。

将待生成的数据结构成为`IntSet`，表示整数集合，可定义以下接口：

In [1]:
import sys
import random

random.seed(2023)

def bigrand():
    return int(sys.maxsize * random.random() +  random.random())

In [2]:
from dataclasses import dataclass
import abc

@dataclass
class IntSetImp:
    maxelements: int
    maxval: int

    @abc.abstractmethod
    def insert(self, t):
        pass

    @abc.abstractmethod
    def size(self):
        pass

    @abc.abstractmethod
    def report(self, v):
        pass

In [3]:
def gensets(m, maxval, setImp):
    S = setImp(m, maxval)
    v = []
    while S.size() < m:
        S.insert(bigrand() % maxval)

    S.report(v)
    return v[:m]

In [4]:
class IntSetSTL(IntSetImp):
    S = set()

    def size(self):
        return len(self.S)

    def insert(self, t):
        self.S.add(t)

    def report(self, v):
        v.extend(list(self.S))

In [5]:
gensets(2, 5, IntSetSTL)

[0, 4]

## 13.2 线性结构

如果事先知道集合的大小，数组是一种比较理想的结构。

In [6]:
class IntSetArray(IntSetImp):

    def __init__(self, maxelements, maxval):
        super().__init__(maxelements, maxval)
        self.x = [0] * (1 + maxelements)
        self.n = 0
        self.x[0] = maxval

    def size(self):
        return self.n

    def insert(self, t):
        i = 0
        while self.x[i] < t:
            i += 1
        if self.x[i] == t:
            return
        for j in range(self.n, i - 1, -1):
            self.x[j + 1] = self.x[j]

        self.x[i] = t
        self.n += 1

    def report(self, v):
        for i in range(self.n + 1):
            v.append(self.x[i])

In [7]:
gensets(2, 5, IntSetArray)

[0, 3]

如果事先不知道集合的大小，链表将是表示集合的首选结构。

In [12]:
class IntSetList(IntSetImp):
    class node:
        def __init__(self, v, p):
            self.val = v
            self.next = p

        def __eq__(self, other):
            return self.val == other.val

    def __init__(self, maxelements, maxval):
        super().__init__(maxelements, maxval)
        self.n = 0
        self.sentinel = self.node(maxval, 0)
        self.head = self.node(maxval, 0)

    def insert(self, t):
        self.head = self.rinsert(self.head, t)

    def rinsert(self, p, t):
        if p.val < t:
            p.next = self.rinsert(p.next, t)
        elif p.val > t:
            p = self.node(t, p)
            self.n += 1
        return p

    def report(self, v):
        p = self.head
        while p != self.sentinel:
            v.append(p.val)
            p = p.next

    def size(self):
        return self.n

In [13]:
gensets(2, 5, IntSetList)

[0, 3]

## 13.3 二分搜索树

二分搜索树的原理：
1. 初始化树的时候将根设置为空。
2. 插入函数遍历这棵树，直到找到该值（搜索终止）或在整棵树中都没有找到该值（插入该结点）。
3. 中序遍历首先处理左子树，然后输出结点本身，最后处理右子树。

In [10]:
class IntSetBST(IntSetImp):
    class node:
        def __init__(self, i):
            self.val = i
            self.left = 0
            self.right = 0

        def __eq__(self, other):
            return self.val == other.val

    def __init__(self, maxelements, maxval):
        super().__init__(maxelements, maxval)
        self.n = 0
        self.root = 0

    def insert(self, t):
        self.root = self.rinsert(self.root, t)

    def report(self, x):
        self.v = x
        self.vn = 0
        self.traverse(self.root)

    def rinsert(self, p, t):
        if isinstance(p, int) and p == 0:
            p = self.node(t)
            self.n += 1
        elif t < p.val:
            # 左子树
            p.left = self.rinsert(p.left, t)
        elif t > p.val:
            p.right = self.rinsert(p.right, t)

        return p

    def traverse(self, p):
        if isinstance(p, int) and p == 0:
            return

        self.traverse(p.left)
        self.v.append(p.val)
        self.traverse(p.right)

    def size(self):
        return self.n

In [11]:
gensets(2, 5, IntSetBST)

[0, 2]

## 13.4 用于整数的结构

箱：结合链表和位向量的优点，在箱序列中放入整数，可以将箱序列看作一种散列，每个箱中的整数用一个有序链表表示，由于整数是均匀分布的，所以每个链表的期望长度都是1。

In [14]:
class IntSetBins(IntSetImp):
    class node:
        def __init__(self, v=0, p=None):
            self.val = v
            self.next = p

        def __eq__(self, other):
            return self.val == other.val

    def __init__(self, maxelements, pmaxval):
        super().__init__(maxelements, pmaxval)
        self.bins = maxelements
        self.maxval = pmaxval
        self.bin = [self.node(self.maxval, 0) for _ in range(self.bins)]
        self.sentinel = self.node(self.maxval, 0)
        self.n = 0

    def insert(self, t):
        i = int(t // (1 + self.maxval / self.bins))
        self.bin[i] = self.rinsert(self.bin[i], t)

    def report(self, v):
        for i in range(self.bins):
            p = self.bin[i]
            while p != self.sentinel:
                v.append(p.val)
                p = p.next

    def rinsert(self, p, t):
        if p.val < t:
            p.next = self.rinsert(p.next, t)
        elif p.val > t:
            p = self.node(t, p)
            self.n += 1

        return p

    def size(self):
        return self.n

In [15]:
gensets(2, 5, IntSetBins)

[0, 2]

## 13.5 原理

- 库的作用：当遇到涉及数据结构的问题时，第一反应是寻找解决问题的工具。
- 空间的重要性：考虑数据的空间，也能优化程序的运行时间。
- 代码调优方法：最显著的改进就是用只分配一个较大内存块的方案来代替通用内存分配，能消除很多开销较大的调用，也使空间的利用更加有效。

## 13.6 一个实际搜索问题（边栏）

**项目背景：**

作者使用Doug McIlroy于1978年写的spell程序对本书各章进行拼写检查。sepll程序是用于表示字典。

**问题分析：**

1. 组成单词列表：用完整版字典和有百万单词的布朗大学英语语料库求交集，得到单词列表。处理专有名词，包含人名、财富500强中的公司名、出版公司的名字、地理名词。添加了动植物的常用名，以及化学、解剖学中的术语，计算机术语等。
2. 采用最具有一般性的解决方法，精选75000个单词，组成列表。
3. 使用词缀分析从单词中去除前后缀，去掉mis-、re-、pre-和-ed。
4. 利用程序的表中包含40条前缀法则和30条后缀法则，使用一个具有1300项例外的“终止列表”来终止符合词缀法则但不正确的猜测。进一步把75000词压缩为30000个单词。

**数据结构表示：**

通过散列表示30000个单词，每个单词用27位表示。

- 方法一：用一个几乎和单词列表一样大的$n$元散列表以及一个把字符串映射为$[0, n)$范围内的整数的散列函数。
- 方法二：选择$n=2^{27}$的散列表，在每个表项中仅存放一位，而不是存放单词链表。当查找单词$w$时，程序访问表中的第$h(w)$位，如果该位为0，则报告单词$w$不在表中，如果为1，则认为$w$在表中。

> 出现错误的概率只有$30000/2^{27}$（约1/4000），即每4000个不正确的单词中只有一个会被认为有效。使用$n=2^{27}$位的字符串白哦是散列表要消耗超过16MB的空间，因此程序仅表示值为1的位。

**性能提升：**

- McIlory的spell程序平均使用13.6位表示每个差值，节省了几百个额外的字节，加快顺序搜索。
- 可以得到64KB的字典，该字典不仅支持快速访问，而且很少出错。